# EDGAR - Reading Information Tables in Text Format - Advanced Text Mining (25 marks)

#### So far, we have collected CIKs for each of the Mutual Funds, then looked up the links of all the 13F HRs and the Information Tables, and identified them either text tables or xml tables. In class, we  obtained the information from the xml tables.  In this assignment we will obtain information from the text file.  These are not as nicely structured as the xml output.   In a csv (HW_Mutual_Fund_Info_Table_Link.csv) you will find a few of these links.  Your goal for this part of the homework is to obtain the link to the text files from the attached file  (HW_Mutual_Fund_Info_Table_Link.csv).  Then you will write a code that will go to the links of all the linked text files, and extract some columns.  Do not use Beutiful Soup for this assignment.   We provide some initial code to guide your initial steps:

#### First we have to collect the links of the text Info Tables in lists:

In [198]:
import urllib.request
import bs4 as bs
from bs4 import SoupStrainer

In [199]:
import urllib

CIK = []
xml_link = []
xml_Name = []
xml_date = []

text_link = []
text_Name = []
text_date = []

input_file = open('HW_Mutual_Fund_Info_Table_Link.csv', 'r')
rows = input_file.readlines()
input_file.close()

# Your Code on Enumerating the Lists.  The result should be three lists, text_link, text_Name,
#and text_date.  Each should have length 122.

for row in range(1, len(rows)):
    column = rows[row].split(",")
    if column[4] == "xml":
        xml_link.append(column[3])
        xml_Name.append(column[1])
        xml_date.append(column[2])
    else:
        text_link.append(column[3])
        text_Name.append(column[1])
        text_date.append(column[2])

#### Keep only the links that correspond to a date after 2010 (Don't inlude 2010, start at 2011).  Hint: you can use the datetime library.

In [200]:
from datetime import datetime as dt

#Use the following list to store the filtered values.
filtered_dates = []
filtered_name = []
filtered_link = []

filtered_text_date = []
filtered_text_name = []
filtered_text_link = []



#Enter code here to to keep only the dates corresponding to after 2010. 

xml_date_converted = [dt.strptime(x,'%Y/%m/%d') for x in xml_date]
xml_year = [x.year for x in xml_date_converted]
for pos in range(0, len(xml_year)):
    if xml_year[pos] > 2010:
        filtered_dates.append(xml_date[pos])
        filtered_name.append(xml_Name[pos])
        filtered_link.append(xml_link[pos])

text_date_converted = [dt.strptime(x,'%Y/%m/%d') for x in text_date]
text_year = [x.year for x in text_date_converted]  
for pos in range(0, len(text_year)):
    if text_year[pos] > 2010:
        filtered_text_date.append(text_date[pos])
        filtered_text_name.append(text_Name[pos])
        filtered_text_link.append(text_link[pos])

#### Your filtered list should now have 8 elements.  These represent 3 mutual funds.   The first one has CIK=1311981 (from the link you can find this at data/1311981 ).  The second was has CIK 813470.  The third one has CIK 1432353. 

['https://www.sec.gov/Archives/edgar/data/1311981/000116204413000513/0001162044-13-000513.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347013000006/0000813470-13-000006.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347013000001/0000813470-13-000001.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000023/0000813470-12-000023.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000019/0000813470-12-000019.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000014/0000813470-12-000014.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000003/0000813470-12-000003.txt',
 'https://www.sec.gov/Archives/edgar/data/1432353/000114420411008428/0001144204-11-008428.txt']

#### Next, for each text link, extract the name of issuer, CUSIP, and the Quantity of shares.  You will also want to keep track of the mutual fund name as well as the filing report date.  

#### Your output file should have 5 columns.  The first is the issue date of the form which can be found in the filtered_date list (this will be repeated for the same form).  The second is the mutual fund name which can be found in the filtered_name list (this will be repeated).  The third, fourth and fifith are the name of issuer, CUSIP, and shares respectively.  Make sure to account for the fact that while some of the text files have the same formatting, others do not.  This means you will have to look through them to make sure your code works for the each text file.

In [201]:
#convert filename to URL links
urls =[]
for link in filtered_link:
    urls.append("https://www.sec.gov" + link)


url_t = []
for t_link in filtered_text_link:
    url_t.append("https://www.sec.gov" + t_link)


In [202]:
mutual_fund_name = []
info_date = []
name_of_issuer = []
CUSIP = []
SHRS_or_PRN = []

In [ ]:
for ind in range(0,len(urls)):
        source = urllib.request.urlopen(urls[ind])
        html = source.read().decode('utf-8')
        soup = BeautifulSoup(html, "html5lib")
        Info = soup.find_all("table", summary = "Form 13F-NT Header Information")
        
        for row in Info[0].find_all('tr')[3:]:
            col = row.find_all('td')
            mutual_fund_name.append(filtered_name[ind])
            info_date.append(filtered_dates[ind])
            name_of_issuer.append(col[0].getText())
            CUSIP.append(col[2].getText())
            SHRS_or_PRN.append(col[4].getText())
            


In [ ]:

for indx, vl in enumerate(url_t):
    Input = urllib.request.urlopen(url_t[indx]).read().decode('utf-8')
    beg = Input.upper().find('NAME OF ISSUER')
    end = Input.find('</TABLE')
    t = Input[beg:end].split('\n')
    
    
    for row in t:
        
        row = row.split("  ")
        row = [x for x in row if x]
  
        if (len(row) > 4):
        
            row[2] = ''.join(c for c in row[2] if c.isalnum())
            row[4] = ''.join(c for c in row[4] if c.isdigit())

            
            df = pd.DataFrame(tuples)
            
            mutual_fund_name.append(filtered_text_name[indx])
            info_date.append(filtered_text_date[indx])

            name_of_issuer.append(row[0])
            CUSIP.append(row[2])
            SHRS_or_PRN.append(row[4])



#### Finally, you write down the lists in the following file:

In [ ]:
# Your code goes here

import csv
from itertools import zip_longest

d = [mutual_fund_name, info_date, name_of_issuer, CUSIP, SHRS_or_PRN]
edgar_file = zip_longest(*d, fillvalue = '')
with open('edgar_file.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(("Mutual_Fund_Name", "Info_Date", "Name_of_Issuer", "CUSIP", "Qty"))
    wr.writerows(edgar_file)

myfile.close()